<h1>4- GCP Auto ML</h1>

In [10]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery

REGION = "us-central1"
DATA_REGION = "US"
PROJECT_ID = "customer-churn-391917"

aiplatform.init(project=PROJECT_ID, location=REGION)
bqclient = bigquery.Client(project=PROJECT_ID)

In [11]:
job=bqclient.query("create schema kkbox2")
job.result()
print(job.state)

DONE


In [13]:
BQ_TABLE = "kkbox.model_member"
BQ_DATASET_NAME="kkbox2"

MODEL_NAME = "model_member_ml"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='BOOSTED_TREE_CLASSIFIER',
    labels = ['is_churn'],
    model_registry='vertex_ai'
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

job = bqclient.query(MODEL_QUERY)
print(f"Job state: {job.state}\nJob Error:{job.errors}")
job.result()
print(job.state)
model = aiplatform.Model(model_name=MODEL_NAME)

print(model.gca_resource)

Job state: RUNNING
Job Error:None
DONE
name: "projects/1043836430795/locations/us-central1/models/model_member_ml"
display_name: "model_member_ml"
supported_deployment_resources_types: DEDICATED_RESOURCES
supported_input_storage_formats: "jsonl"
supported_input_storage_formats: "bigquery"
supported_input_storage_formats: "csv"
supported_input_storage_formats: "tf-record"
supported_input_storage_formats: "tf-record-gzip"
supported_input_storage_formats: "file-list"
supported_output_storage_formats: "jsonl"
supported_output_storage_formats: "bigquery"
create_time {
  seconds: 1693347996
  nanos: 604032000
}
update_time {
  seconds: 1693348103
  nanos: 247988000
}
etag: "AMEw9yPtBfVyD8H01zNU16Gx3P7gP4BXf1xO7qb12rzWoT9thUSbZExdYqtDeJwZYbs="
explanation_spec {
  parameters {
    sampled_shapley_attribution {
      path_count: 5
    }
  }
  metadata {
    inputs {
      key: "activity"
      value {
      }
    }
    inputs {
      key: "actual_amount_paid"
      value {
      }
    }
    in

In [14]:
ENDPOINT_DISPLAY_NAME = "member-model-endpoint"
endpoint = aiplatform.Endpoint.create(
    display_name=ENDPOINT_DISPLAY_NAME,
    project=PROJECT_ID,
    location=REGION,
)

print(endpoint.display_name)
print(endpoint.resource_name)

Creating Endpoint
Create Endpoint backing LRO: projects/1043836430795/locations/us-central1/endpoints/7423130653580853248/operations/6206249114477264896
Endpoint created. Resource name: projects/1043836430795/locations/us-central1/endpoints/7423130653580853248
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/1043836430795/locations/us-central1/endpoints/7423130653580853248')
member-model-endpoint
projects/1043836430795/locations/us-central1/endpoints/7423130653580853248


In [15]:

DEPLOYED_NAME = "bqml-lr-member"

model.deploy(endpoint=endpoint, deployed_model_display_name=DEPLOYED_NAME)

print(model.display_name)
print(model.resource_name)

Deploying model to Endpoint : projects/1043836430795/locations/us-central1/endpoints/7423130653580853248
Using default machine_type: n1-standard-2
Deploy Endpoint model backing LRO: projects/1043836430795/locations/us-central1/endpoints/7423130653580853248/operations/1945843866984775680
Endpoint model deployed. Resource name: projects/1043836430795/locations/us-central1/endpoints/7423130653580853248
model_member_ml
projects/1043836430795/locations/us-central1/models/model_member_ml


In [16]:
sql_ml_predict = f"""SELECT * FROM ML.PREDICT(MODEL `{PROJECT_ID}.{BQ_DATASET_NAME}.{MODEL_NAME}`, 
(SELECT
      *
    FROM
      `{BQ_TABLE}` LIMIT 10))"""

job = bqclient.query(sql_ml_predict)
prediction_result = job.result().to_arrow().to_pandas()
     

In [22]:
display(prediction_result.head())

,predicted_is_churn,predicted_is_churn_probs,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,...,activity,sum_num_25,sum_num_50,sum_num_75,sum_num_985,sum_num_100,sum_num_unq,total_secs,extracted_label,label_equals_churn
0,1,"[{'label': 1, 'prob': 0.9696462154388428}, {'l...",V0QKACbfelvDGa8PwWb3eJK5SZPM2ps/TXflgE/I/yM=,1,13.0,25.0,NAN,7.0,2013-01-22,41,...,23,162,43,40,45,502,486,142461.208,1,True
1,1,"[{'label': 1, 'prob': 0.966254472732544}, {'la...",w9uv6HXTzDgFHbTpXyYatmbSY0CLOsQUP1cCSlSgpfE=,1,15.0,0.0,female,7.0,2013-04-20,41,...,28,50,18,12,12,857,915,211659.246,1,True
2,1,"[{'label': 1, 'prob': 0.9636967182159424}, {'l...",5zpIN5OvTtMA0/VaylZ2u6rKzwsDg0BrEDkSZjb+IVA=,1,22.0,37.0,male,7.0,2011-10-11,41,...,28,101,41,25,55,922,907,250189.864,1,True
3,1,"[{'label': 1, 'prob': 0.9448596835136414}, {'l...",tyCe3lwEf2JdRPQKk2ik8O/Uo4db7F3oLkn+nXBshv8=,1,13.0,25.0,female,7.0,2015-02-11,41,...,23,521,147,39,26,456,1028,129333.852,1,True
4,1,"[{'label': 1, 'prob': 0.7790024876594543}, {'l...",HDdBsA7I7+eB51OTwWKhiCemyuagDXelHT7DtkdS+AA=,1,15.0,29.0,female,7.0,2013-08-02,38,...,16,17,16,11,10,126,174,39819.177,1,True


In [23]:
df=prediction_result

# Finding rows where 'extracted_label' is equal to 'is_churn'
df['label_equals_churn'] = df['predicted_is_churn'] == df['is_churn']

# Calculating the percentage of rows where 'extracted_label' is equal to 'is_churn'
percentage_equal = (df['label_equals_churn'].sum() / len(df)) * 100

print(f"Percentage of rows where predicted label and actual churn are equal: {percentage_equal}%")

Percentage of rows where predicted label and actual churn are equal: 100.0%
